In [1]:
from fpdf import FPDF
from PyPDF2 import PdfFileMerger, PdfFileReader
import os

class PDF(FPDF):
    def section_title(self, label):
        # Arial 12
        self.set_font('Arial', '', 12)
        # Background color
        self.set_fill_color(200, 220, 255)
        # Title
        self.cell(0, 6, 'Section : %s' % (label), 0, 1, 'L', 1)
        # Line break
        self.ln(4)
        
    def section_body(self, results_dir, name):
        # Read text file
        file_path = os.path.join(results_dir, name)
        with open(file_path, 'rb') as fh:
            txt_content = fh.read().splitlines()
            
        # Courier 8
        self.set_font('Courier', '', 8)
        # self.set_font('Times', '', 12)
        for txt_line in txt_content:
            self.cell(0,0, txt_line)
            self.ln(3)

    def print_section(self, title, results_dir, name):
        self.add_page()
        self.section_title(title)
        self.section_body(results_dir, name)

In [2]:
title = 'FSI-cells study'

test_name = 'neuroM_morph_softChecks'
model_results = 'cells_str-fs_excerpt'
val_results_dir='./validation_results'
val_results_path = os.path.join(val_results_dir, test_name, model_results)

val_reports_dir='./validation_reports'
val_reports_path = os.path.join(val_reports_dir, test_name, model_results)
if not os.path.exists(val_reports_path):
    os.makedirs(val_reports_path)

# pdf.set_title(title)
# pdf.set_author('Pedro Garcia-Rodriguez')

merger = PdfFileMerger()
dir_names = os.listdir(val_results_path) # Cells dirs

for dir_name in sorted(dir_names):

    dir_path=os.path.join(val_results_path,dir_name)
    dir_date = os.listdir(dir_path)[0]
    dir_date_path=os.path.join(dir_path, dir_date)
    file_names=os.listdir(dir_date_path) # Cell files
    
    pdf = PDF()
    for file_name in sorted(file_names):
        if file_name.endswith(('.json', '.txt')):
            pdf.print_section(file_name, dir_date_path, file_name)
        elif file_name.endswith('.pdf'):
            pdf_path = os.path.join(dir_date_path, file_name)
    text_pdf_path = os.path.join(val_reports_path, 'FSI_cell_val_report_txt.pdf')
    pdf.output(text_pdf_path, 'F')
    merger.append(PdfFileReader(text_pdf_path), 'rb')
    merger.append(PdfFileReader(pdf_path), 'rb')
    
report_pdf_path = os.path.join(val_reports_path, 'FSI_cells_validation_report.pdf')
merger.write(report_pdf_path)

os.remove(text_pdf_path) # Remove last cell pdf. file

In [ ]:
title = 'Population study'

test_name = 'neuroM_morph_softChecks'
model_results = 'population_str-fs_excerpt/20180529-100621'
val_results_dir='./validation_results'
val_results_path = os.path.join(val_results_dir, test_name, model_results)

val_reports_dir='./validation_reports'
val_reports_path = os.path.join(val_reports_dir, test_name, model_results)
if not os.path.exists(val_reports_path):
    os.makedirs(val_reports_path)
    
pdf = PDF()
pdf.set_title(title)
pdf.set_author('Pedro Garcia-Rodriguez')
pdf.print_section('prediction_summary_FSI_mean.json', val_results_path, 'prediction_summary_FSI_mean.json')
pdf.print_section('validation_summary_FSI_mean.txt', val_results_path, 'results_summary_FSI_mean.txt')
pdf.print_section('scores_summary_FSI_mean.json', val_results_path, 'scores_summary_FSI_mean.json')


text_pdf_path = os.path.join(val_reports_path, 'FSI_population_val_report_txt.pdf')     
pdf.output(text_pdf_path, 'F')

merger = PdfFileMerger()
merger.append(PdfFileReader(text_pdf_path), 'rb')
file_paths = os.listdir(val_results_path)
for file_path in file_paths:
    if file_path.endswith('.pdf'):
        pdf_path = os.path.join(val_results_path, file_path)
        merger.append(PdfFileReader(pdf_path), 'rb')

report_pdf_path = os.path.join(val_reports_path, 'FSI_population_validation_report.pdf')
merger.write(report_pdf_path)
os.remove(text_pdf_path)